# Revisión de datos de los archivos csv de Open Contracting Partnership. Data Set del 2018.

Usando el buscador de OpenContracting se buscaron datos sobre Chile (buscador: https://data.open-contracting.org/es/search/). Se decidió descargarlos archivos de los años 2018 y 2021, para tener una muestra de datos que contenga información sobre licitaciones en un rango de tiempo cercano a estas fechas.

In [1]:
import pandas as pd
import numpy as np
import datetime
from os import path

In [2]:
%cd Chile
%cd 2018
%ls
%cd ..
%cd ..

c:\Users\camil\OneDrive\Documentos\GitHub\Proyecto-IMT-2200-1-Grupo-14\Datos y trabajo con muestras\Chile
c:\Users\camil\OneDrive\Documentos\GitHub\Proyecto-IMT-2200-1-Grupo-14\Datos y trabajo con muestras\Chile\2018
 El volumen de la unidad C no tiene etiqueta.
 El n�mero de serie del volumen es: 82E3-D5B2

 Directorio de c:\Users\camil\OneDrive\Documentos\GitHub\Proyecto-IMT-2200-1-Grupo-14\Datos y trabajo con muestras\Chile\2018

22-11-2024  23:56    <DIR>          .
23-11-2024  19:18    <DIR>          ..
22-11-2024  18:54        64.521.477 awards.csv
18-11-2024  15:29       137.878.898 awards_documents.csv
18-11-2024  15:29       172.974.592 awards_items.csv
22-11-2024  21:23        30.901.082 awards_suppliers.csv
19-08-2024  11:45       165.869.422 main.csv
22-11-2024  20:30       370.411.787 parties.csv
22-11-2024  23:33       243.498.032 parties_additionalIdentifiers.csv
18-11-2024  15:30       287.507.728 tender_items.csv
18-11-2024  15:27       103.927.462 tender_tenderers.csv

## Revision de main.csv

In [3]:
camino = path.join("Chile","2018","main.csv")
df_main = pd.read_csv(camino, sep=",",low_memory=False)

In [4]:
df_respaldo = df_main

In [5]:
df_main.head()

,_link,id,tag,date,ocid,language,initiationType,buyer_id,buyer_name,tender_id,...,tender_tenderPeriod_startDate,tender_tenderPeriod_durationInDays,tender_enquiryPeriod_endDate,tender_enquiryPeriod_startDate,tender_enquiryPeriod_durationInDays,tender_procuringEntity_id,tender_procuringEntity_name,tender_value_amount,tender_value_currency,tender_value_unitOfAccount
0,id-0.0,ocds-70d2nz-4548-142-L117-2018-12-31T11:27:18Z,compiled,2018-12-31T11:27:18Z,ocds-70d2nz-4548-142-L117,es,tender,CL-MP-5464,Ilustre Municipalidad de Quintero | UNIDAD NORMAL,4548-142-L117,...,2017-12-07T16:17:35Z,6.0,2017-12-10T20:04:00Z,2017-12-07T20:04:00Z,3.0,CL-MP-5464,Ilustre Municipalidad de Quintero | UNIDAD NORMAL,NaN,NaN,NaN
1,id-0.1,ocds-70d2nz-2409-1129-L117-2018-12-31T11:28:26Z,compiled,2018-12-31T11:28:26Z,ocds-70d2nz-2409-1129-L117,es,tender,CL-MP-3401,Municipalidad de Los Angeles | MUNICIPALIDAD D...,2409-1129-L117,...,2017-12-18T17:58:24Z,8.0,2017-12-20T14:30:00Z,2017-12-18T19:01:00Z,2.0,CL-MP-3401,Municipalidad de Los Angeles | MUNICIPALIDAD D...,300000.0,CLP,NaN
2,id-0.2,ocds-70d2nz-2961-811-L117-2018-12-31T11:29:05Z,compiled,2018-12-31T11:29:05Z,ocds-70d2nz-2961-811-L117,es,tender,CL-MP-3939,ILUSTRE MUNICIPALIDAD DE LA GRANJA | Adquisici...,2961-811-L117,...,2017-12-19T13:02:43Z,14.0,2017-12-28T17:00:00Z,2017-12-20T09:00:00Z,8.0,CL-MP-3939,ILUSTRE MUNICIPALIDAD DE LA GRANJA | Adquisici...,1452000.0,CLP,NaN
3,id-0.3,ocds-70d2nz-4858-195-L117-2018-12-31T11:21:15Z,compiled,2018-12-31T11:21:15Z,ocds-70d2nz-4858-195-L117,es,tender,CL-MP-5772,Municipalidad de Tirúa | Ilustre Municipalidad...,4858-195-L117,...,2017-12-15T12:23:25Z,6.0,2017-12-18T15:01:00Z,2017-12-15T15:52:00Z,3.0,CL-MP-5772,Municipalidad de Tirúa | Ilustre Municipalidad...,900000.0,CLP,NaN
4,id-0.4,ocds-70d2nz-2409-975-LE17-2018-12-31T11:20:54Z,compiled,2018-12-31T11:20:54Z,ocds-70d2nz-2409-975-LE17,es,tender,CL-MP-3401,Municipalidad de Los Angeles | MUNICIPALIDAD D...,2409-975-LE17,...,2017-12-05T17:11:36Z,6.0,2017-12-06T17:30:00Z,2017-12-05T17:15:00Z,1.0,CL-MP-3401,Municipalidad de Los Angeles | MUNICIPALIDAD D...,6000000.0,CLP,NaN


In [6]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244874 entries, 0 to 244873
Data columns (total 30 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   _link                                244874 non-null  object 
 1   id                                   244874 non-null  object 
 2   tag                                  244874 non-null  object 
 3   date                                 244874 non-null  object 
 4   ocid                                 244874 non-null  object 
 5   language                             244874 non-null  object 
 6   initiationType                       244874 non-null  object 
 7   buyer_id                             244874 non-null  object 
 8   buyer_name                           244874 non-null  object 
 9   tender_id                            244874 non-null  object 
 10  tender_procurementMethodDetails      244874 non-null  object 
 11  tender_title 

In [7]:
df_main.columns

Index(['_link', 'id', 'tag', 'date', 'ocid', 'language', 'initiationType',
       'buyer_id', 'buyer_name', 'tender_id',
       'tender_procurementMethodDetails', 'tender_title', 'tender_status',
       'tender_procurementMethod', 'tender_hasEnquiries', 'tender_description',
       'tender_awardPeriod_endDate', 'tender_awardPeriod_startDate',
       'tender_awardPeriod_durationInDays', 'tender_tenderPeriod_endDate',
       'tender_tenderPeriod_startDate', 'tender_tenderPeriod_durationInDays',
       'tender_enquiryPeriod_endDate', 'tender_enquiryPeriod_startDate',
       'tender_enquiryPeriod_durationInDays', 'tender_procuringEntity_id',
       'tender_procuringEntity_name', 'tender_value_amount',
       'tender_value_currency', 'tender_value_unitOfAccount'],
      dtype='object')

In [8]:
df_main["tender_procurementMethodDetails"].unique()

array(['Licitación Pública Menor a 100 UTM (L1)',
       'Licitación Pública Entre 100 y 1000 UTM (LE)',
       'Licitación Pública Mayor 1000 UTM (LP)',
       'Licitación Pública Mayor a 5000 (LR)',
       'Licitación Pública entre a 2000 y 5000 UTM (LQ)',
       'Licitación Privada Mayor a 5000 (I2)',
       'Licitación Privada Menor a 100 UTM.',
       'Licitación Privada Mayor a 1000 UTM',
       'Licitación Privada entre 100 y 1000 UTM.',
       'Licitación Privada entre a 2000 y 5000 UTM (H2)',
       'Licitación Pública Servicios personales especializados (LS)'],
      dtype=object)

Se utilizan la pagina <<https://api.mercadopublico.cl/modules/Licitacion.aspx>> para crear el diccionario.

In [9]:

tipos_licitaciones ={'Licitación Pública Menor a 100 UTM (L1)': "L1",
       'Licitación Pública Entre 100 y 1000 UTM (LE)': "LE",
       'Licitación Pública Mayor 1000 UTM (LP)': "LP",
       'Licitación Pública entre a 2000 y 5000 UTM (LQ)':"LQ",
       'Licitación Pública Mayor a 5000 (LR)' :"LR",
       'Licitación Privada entre a 2000 y 5000 UTM (H2)':"H2",
       'Licitación Pública Servicios personales especializados (LS)':"LS",
       "Licitación Privada por Licitación Pública anterior sin oferentes (A1)" : "A1",
       "Licitación Privada por Remanente de Contrato anterior (B1)": "B1",
       "Licitación Privada por Convenios con Personas Jurídicas Extranjeras fuera del Territorio Nacional (E1)": "E1",
       "Licitación Privada por Servicios de Naturaleza Confidencial (F1)": "F1",
       "Licitación Privada por otras causales, excluidas de la ley de Compras (J1)": "J1",
       "Licitación Privada entre 100 y 1000 UTM" :"CO", "Licitación Privada Mayor a 1000 UTM": "B2",
       "Trato Directo por Producto de Licitación Privada anterior sin oferentes o desierta": "A2",
       "Trato Directo por Proveedor Único (D1)": "D1","Licitación Privada Menor a 100 UTM" : "E2", "Trato Directo (Cotización) (C2)" : "C2",
       "Compra Directa (Orden de compra) (C1)" : "C1", "Trato Directo (Cotización) (F2)" :"F2", "Compra Directa (Orden de compra) (F3)" : "F3",
       "Directo (Cotización) (G2)" : "G2", "Compra Directa (Orden de compra) (G1)" : "G1",
       "Orden de Compra menor a 3 UTM (R1)" : "R1", "Orden de Compra sin Resolución (CA)" : "CA", 
       "Orden de Compra proveniente de adquisición sin emisión automática de OC (SE)" : "SE"}


df_main["tender_procurementMethodDetails"] = df_main["tender_procurementMethodDetails"].map(tipos_licitaciones)
df_main["tender_procurementMethodDetails"] = df_main["tender_procurementMethodDetails"].astype("category")

In [10]:
df_main["buyer_id"]

0           CL-MP-5464
1           CL-MP-3401
2           CL-MP-3939
3           CL-MP-5772
4           CL-MP-3401
              ...     
244869    CL-MP-572923
244870      CL-MP-4543
244871      CL-MP-1321
244872      CL-MP-4482
244873      CL-MP-3301
Name: buyer_id, Length: 244874, dtype: object

In [11]:
def dejar_rut(texto:str) -> str:
    return int(texto.split("-")[2])

df_main["buyer_id"] = df_main["buyer_id"].apply(dejar_rut)

In [12]:
df_main["buyer_id"]

0           5464
1           3401
2           3939
3           5772
4           3401
           ...  
244869    572923
244870      4543
244871      1321
244872      4482
244873      3301
Name: buyer_id, Length: 244874, dtype: int64

In [13]:
df_main.drop(columns=['id', 'tag', 'date', 'ocid', 'language', 'buyer_name', 'tender_title', 'tender_description', 'tender_procuringEntity_id'], inplace=True)

In [14]:
df_main.loc[df_main['tender_enquiryPeriod_startDate'] == "3"]

,_link,initiationType,buyer_id,tender_id,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,tender_tenderPeriod_endDate,tender_tenderPeriod_startDate,tender_tenderPeriod_durationInDays,tender_enquiryPeriod_endDate,tender_enquiryPeriod_startDate,tender_enquiryPeriod_durationInDays,tender_procuringEntity_name,tender_value_amount,tender_value_currency,tender_value_unitOfAccount


### Transformación de datos.

In [15]:
df_main["tender_enquiryPeriod_startDate"] = pd.to_datetime(df_main['tender_enquiryPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main["tender_enquiryPeriod_endDate"] = pd.to_datetime(df_main['tender_enquiryPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main['tender_tenderPeriod_startDate'] = pd.to_datetime(df_main['tender_tenderPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')

Al parecer hay un error, una de las filas tiene por valor a en esta columna, eliminemosla.

In [16]:
df_main = df_main.drop(df_main[df_main["tender_tenderPeriod_endDate"] == "a"].index)
df_main["tender_tenderPeriod_endDate"] = pd.to_datetime(df_main['tender_tenderPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main['tender_awardPeriod_startDate'] = pd.to_datetime(df_main['tender_awardPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')

Hay un error en el formato de una fila con contenido <5017-05-05T17:00:00Z>, se eliminará.

In [17]:
df_main.drop(df_main[df_main["tender_awardPeriod_endDate"] == "5017-05-05T17:00:00Z"].index, axis=0, inplace=True)
df_main['tender_awardPeriod_endDate'] = pd.to_datetime(df_main['tender_awardPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main['tender_enquiryPeriod_durationInDays'] = df_main['tender_enquiryPeriod_endDate'] - df_main['tender_enquiryPeriod_startDate']
df_main['tender_awardPeriod_durationInDays'] = df_main['tender_awardPeriod_endDate'] - df_main['tender_awardPeriod_startDate']
df_main['tender_tenderPeriod_durationInDays'] = df_main['tender_tenderPeriod_endDate'] - df_main['tender_tenderPeriod_startDate']

### Filtro según fecha

Aqui realizaremos un filtro de las licitaciones en base a cuando se iniciaron y cuando terminaron respecto a lo relacionado al 2018. En este caso seleccionaremos las licitaciones cuyo periodo de envio de aplicaciones para determinar la adjudicación comenzó despúes que el 01/09/2017 y la finalización del proceso de adjudicación antes del 01/01/2019.

In [18]:
fecha_filtro = '2017-09-01'
df_filtrado = df_main.loc[df_main['tender_tenderPeriod_startDate'] >= fecha_filtro]
fecha_filtro = '2019-01-01'
df_filtrado = df_filtrado.loc[df_filtrado['tender_awardPeriod_endDate'] < fecha_filtro]

In [19]:
df_filtrado.reset_index(drop=True, inplace=True)
df_filtrado.rename(columns={"tender_value_amount":"estimated_cost", "tender_value_currency": "estimated_cost_currency", "tender_value_unitOfAccount": "estimated_cost_unitOfAccount"}, inplace=True )
df_filtrado["estimated_cost_currency"] = df_filtrado["estimated_cost_currency"].replace(np.nan, "No")

In [20]:
def crear_unit_value_monetary_unit(indice:int, comparador="No") -> str:
    if df_filtrado.iloc[indice]["estimated_cost_currency"] == comparador:
        return df_filtrado.iloc[indice]["estimated_cost_unitOfAccount"]
    return df_filtrado.iloc[indice]["estimated_cost_currency"]

df_filtrado["estimated_cost_monetary_unit"] = df_filtrado.index.map(crear_unit_value_monetary_unit)

In [21]:
df_filtrado["estimated_cost_monetary_unit"].unique()

array([nan, 'CLP', 'CLF', 'USD', 'UTM', 'EUR'], dtype=object)

In [22]:
df_filtrado.drop(columns=["estimated_cost_currency","estimated_cost_unitOfAccount"],inplace=True)
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31068 entries, 0 to 31067
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype          
---  ------                               --------------  -----          
 0   _link                                31068 non-null  object         
 1   initiationType                       31068 non-null  object         
 2   buyer_id                             31068 non-null  int64          
 3   tender_id                            31068 non-null  object         
 4   tender_procurementMethodDetails      30763 non-null  category       
 5   tender_status                        31068 non-null  object         
 6   tender_procurementMethod             31068 non-null  object         
 7   tender_hasEnquiries                  31068 non-null  bool           
 8   tender_awardPeriod_endDate           31068 non-null  datetime64[ns] 
 9   tender_awardPeriod_startDate         31068 non-null  datetime64[ns] 
 10

## Revision awards.csv

In [23]:
camino = path.join("Chile","2018","awards.csv")
df_awards = pd.read_csv(camino, sep=",")

In [24]:
df_awards.head(8)

,_link,_link_main,id,date,description,title,status,value_amount,value_currency,value_unitOfAccount
0,id-0.0.awards.0,id-0.0,8270925,2017-12-23T13:23:24Z,DIDECO - NAVIDAD COMUNAL - JUEGOS INFLABLES CO...,DIDECO - NAVIDAD COMUNAL - JUEGOS INFLABLES,active,690000.0,CLP,NaN
1,id-0.1.awards.0,id-0.1,8274205,NaN,CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METAL...,CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METÁL...,unsuccessful,NaN,NaN,NaN
2,id-0.2.awards.0,id-0.2,8275841,2018-02-02T11:16:54Z,sm80 sp3565 servicios generales,Reparacion de mamparas,active,870000.0,CLP,NaN
3,id-0.3.awards.0,id-0.3,8254526,2017-12-26T18:19:33Z,VISITA A HUERTO ORGANICO DE BERRIES HUBICADO E...,VISTA A HUERTO ORGANICO DE BERRIES HUBICADO EN...,unsuccessful,NaN,NaN,NaN
4,id-0.4.awards.0,id-0.4,8253535,2017-12-21T18:17:23Z,CAPACITACIÓN PARA DOCENTES DE LA ESC. F-929 - ...,CAPACITACIÓN PARA DOCENTES DE LA ESC. F-929 - ...,active,6000000.0,CLP,NaN
5,id-0.5.awards.0,id-0.5,8277247,2018-01-16T10:56:54Z,Adquisicion requerida por el Sr. Juan Pablo Re...,Adquisición programa rehabilitación integral e...,active,347043.0,CLP,NaN
6,id-0.6.awards.0,id-0.6,8274542,2017-12-28T21:54:24Z,SE REQUIERE LA ADQUISICION DE REACTIVOS PARA L...,REACTIVOS PARA LABORATORIO,active,12219604.0,CLP,NaN
7,id-0.7.awards.0,id-0.7,8271070,2017-12-29T11:54:57Z,PROGRAMA ATENCIÓN INTEGRAL FAMILIAR 24 HORAS,TERMO-VENTILADOR Y ESTUFA A GAS (N°1017) COM.S...,active,191241.0,CLP,NaN


In [25]:
df_no_nan_un_ac = df_awards.dropna(subset=["value_unitOfAccount"])[["value_currency", "value_unitOfAccount", "status"]]
df_unit_of_account = df_no_nan_un_ac.loc[df_no_nan_un_ac["value_currency"].isna()]

In [26]:
df_unit_of_account["value_unitOfAccount"].describe()

count      83
unique      1
top       UTM
freq       83
Name: value_unitOfAccount, dtype: object

In [27]:
df_no_nan_un_ac["value_unitOfAccount"].describe()

count      84
unique      2
top       UTM
freq       83
Name: value_unitOfAccount, dtype: object

In [28]:
df_awards["value_currency"] = df_awards["value_currency"].replace(np.nan, "No")

In [29]:
def crear_unit_value_monetary_unit(indice:int, comparador="No") -> str:
    if df_awards.iloc[indice]["value_currency"] == comparador:
        return df_awards.iloc[indice]["value_unitOfAccount"]
    return df_awards.iloc[indice]["value_currency"]

df_awards["value_monetary_unit"] = df_awards.index.map(crear_unit_value_monetary_unit)

In [30]:
df_awards["value_monetary_unit"].unique()

array(['CLP', nan, 'CLF', 'USD', 'UTM', 'EUR'], dtype=object)

In [31]:
df_awards.drop(columns=['id', 'description', 'title', "value_unitOfAccount", "value_currency","value_unitOfAccount"],inplace=True)

In [32]:
df_awards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240021 entries, 0 to 240020
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   _link                240021 non-null  object 
 1   _link_main           240021 non-null  object 
 2   date                 210324 non-null  object 
 3   status               240021 non-null  object 
 4   value_amount         177451 non-null  float64
 5   value_monetary_unit  177451 non-null  object 
dtypes: float64(1), object(5)
memory usage: 11.0+ MB


## Revisión de awards_items.csv

In [33]:
camino = path.join("Chile","2018","awards_items.csv") 
df_awards_items = pd.read_csv(camino, sep=",")

In [34]:
df_awards_items.columns

Index(['_link', '_link_awards', '_link_main', 'id', 'description', 'quantity',
       'unit_name', 'unit_value_amount', 'unit_value_currency',
       'classification_id', 'classification_uri', 'classification_scheme',
       'unit_value_unitOfAccount'],
      dtype='object')

In [35]:
df_awards_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695151 entries, 0 to 695150
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   _link                     695151 non-null  object 
 1   _link_awards              695151 non-null  object 
 2   _link_main                695151 non-null  object 
 3   id                        695151 non-null  int64  
 4   description               693046 non-null  object 
 5   quantity                  695151 non-null  float64
 6   unit_name                 695151 non-null  object 
 7   unit_value_amount         695151 non-null  float64
 8   unit_value_currency       695022 non-null  object 
 9   classification_id         695151 non-null  int64  
 10  classification_uri        695151 non-null  object 
 11  classification_scheme     695151 non-null  object 
 12  unit_value_unitOfAccount  129 non-null     object 
dtypes: float64(2), int64(2), object(9)
memory us

In [36]:
df_awards_items["unit_value_unitOfAccount"].describe()

count     129
unique      1
top       UTM
freq      129
Name: unit_value_unitOfAccount, dtype: object

In [37]:
df_awards_items.loc[df_awards_items["unit_value_currency"].isna()]["unit_value_unitOfAccount"].describe()

count     129
unique      1
top       UTM
freq      129
Name: unit_value_unitOfAccount, dtype: object

Podemos observar que para los valores de <unit_value_currency> que son NaN existe un dato en <unit_value_unitOfAccount>, por lo que crearemos una nueva columna llamada <unit_value_monetary_unit> que tenga los valores de ambas columna según correspondan.

In [38]:
df_awards_items["unit_value_currency"] = df_awards_items["unit_value_currency"].replace(np.nan, "No")

In [39]:
def crear_unit_value_monetary_unit(indice:int, comparador="No") -> str:
    if df_awards_items.iloc[indice]["unit_value_currency"] == comparador:
        return df_awards_items.iloc[indice]["unit_value_unitOfAccount"]
    return df_awards_items.iloc[indice]["unit_value_currency"]

df_awards_items["unit_value_monetary_unit"] = df_awards_items.index.map(crear_unit_value_monetary_unit)

KeyboardInterrupt: 

In [ ]:
df_awards_items["unit_value_monetary_unit"].unique()

In [ ]:
df_awards_items.drop(columns=['classification_uri', 'classification_scheme','unit_value_unitOfAccount', "unit_value_currency", "description"], inplace=True)

## Revisión de awards_suppliers.csv

In [ ]:
camino = path.join("Chile","2018","awards_suppliers.csv")
df_awards_supplier = pd.read_csv(camino, sep=",")

In [ ]:
df_awards_supplier.info()

In [ ]:
df_awards_supplier.head()

In [ ]:
def dejar_rut(texto:str) -> str:
    return int(texto.split("-")[2])

In [ ]:
df_awards_supplier["id"] = df_awards_supplier["id"].apply(dejar_rut)

In [ ]:
df_awards_supplier.info()

## Revisión de awards_documents.csv

In [ ]:
camino = path.join("Chile","2018", "awards_documents.csv")
df_award_documents = pd.read_csv(camino, sep=",")
df_award_documents.head()

In [ ]:
df_award_documents.info()

No se utilizará este DataFrame, puesto que no contiene información relevante para lo que se busca resolver.

## Revisión de parties.csv

In [ ]:
camino = path.join("Chile","2018", "parties.csv")
df_parties = pd.read_csv(camino, sep=",")
df_parties.head()

In [ ]:
df_parties.info()

In [ ]:
df_parties.drop(columns=['address_streetAddress' "id", 'identifier_scheme','identifier_legalName', 'contactPoint_name', 'contactPoint_email','contactPoint_telephone', 'contactPoint_faxNumber'], inplace=True)

In [ ]:
df_parties.info()

Se eliminan estas columnas porque ofrecen información que no es de interes, ni se puede utilizar para obtener información en alguna de las paginas relacionadas al dataset.

In [ ]:
df_parties["roles"].unique()

In [ ]:
def separar_roles(texto:str) -> list:
    if "," in texto:
        return texto.split(",")
    else:
        return [texto, np.nan]
    
df_parties["roles"] = df_parties["roles"].apply(separar_roles)

In [ ]:
df_parties["rol 1"] = df_parties.index.map(lambda x: df_parties.loc[x]["roles"][0])

In [ ]:
df_parties["rol 2"] = df_parties.index.map(lambda x: df_parties.loc[x]["roles"][1])

In [ ]:
df_parties.head()

In [ ]:
df_G = df_parties.dropna(subset=["address_region"])
df_G = df_G[df_G["address_countryName"].isna()]

In [ ]:
df_G["address_region"].unique()

In [ ]:
df_parties["address_countryName"] = df_parties["address_countryName"].replace(np.nan, "no")
df_parties["address_region"] = df_parties["address_region"].replace(np.nan, "no")

In [ ]:
def dar_pais_segun_region(indice:int, comparador="no") -> str:
    if df_parties.iloc[indice]["address_countryName"] == comparador:
        if df_parties.iloc[indice]["address_region"] in set(['Región de Valparaíso ', 'Región del Biobío ','Región Metropolitana de Santiago','Región de Magallanes y de la Antártica','Región del Libertador General Bernardo O´Higgins','Región de Atacama ', 'Región de Coquimbo ',
       'Región de la Araucanía ', 'Región de los Lagos ','Región del Maule ', 'Región de Los Ríos','Región de Antofagasta ', 'Región de Tarapacá  ','Región de Arica y Parinacota','Región Aysén del General Carlos Ibáñez del Campo']):
            return ("Chile")
        else:
            return ("no")
    else:
        return df_parties.iloc[indice]["address_countryName"]

In [ ]:
def dar_region_segun_pais(indice:int, comparador="no") -> str:
    if df_parties.iloc[indice]["address_region"] == comparador:
        if df_parties.iloc[indice]["address_countryName"] != "Chile" and df_parties.iloc[indice]["address_countryName"] != "no":
            return("Extranjero")
        else:
            return np.nan
    else:
        return df_parties.iloc[indice]["address_region"]

In [ ]:
df_parties["address_countryName"] = df_parties.index.map(dar_pais_segun_region)

In [ ]:
df_parties["address_region"] = df_parties.index.map(dar_region_segun_pais)

In [ ]:
df_parties["address_region"].unique()

In [ ]:
df_parties["address_countryName"].replace("no", np.nan, inplace=True)
df_parties["address_countryName"].unique()

In [ ]:
df_parties.info()

In [ ]:
df_parties.drop(columns=["roles"], inplace=True)

## Revisión de parties_additionalIdentifiers.csv

In [ ]:
camino = path.join("Chile","2018", "parties_additionalIdentifiers.csv")
df_parties_add = pd.read_csv(camino, sep=",")
df_parties_add.head()

In [ ]:
df_parties_add.info()

In [ ]:
df_parties_add.iloc[3]["uri"]

Los links de uri no aportan información extra, por lo no se mantendran.

In [ ]:
df_parties_add.drop(columns=["uri", "scheme", "_link_main"],inplace=True)

### Revisión de tender_items.csv

In [ ]:
camino = path.join("Chile","2018", "tender_items.csv")
df_tender_item = pd.read_csv(camino, sep=",")
df_tender_item.head()

In [ ]:
df_tender_item.info()

In [ ]:
df_tender_item.drop(columns=["classification_uri"], inplace=True)

In [ ]:
df_tender_item.info()

### Revisión de tender_tenderers.csv

In [ ]:
camino = path.join("Chile","2018", "tender_tenderers.csv")
df_tender_tenderers = pd.read_csv(camino, sep=",")

In [ ]:
df_tender_tenderers.head()

In [ ]:
df_tender_tenderers.info()

In [ ]:
df_tender_tenderers["id"] = df_tender_tenderers["id"].apply(dejar_rut)

In [ ]:
df_tender_tenderers.info()

### Clasificación con UNSPCS.

In [ ]:
df_clasificaciones = pd.read_excel("unspcs-clasificador-de-bienes-y-servicios-de-naciones-unidas-en-espanol.xlsx", skiprows=5)
df_clasificaciones.head(10)

El dataframe df_clasificaciones corresponde a un conjunto de datos que posee el significado de cada objeto solicitados según la codificación UNSPSC, por lo que se hara merge con el df_main para saber que se esta comprando en cada licitación, se desecharán algunas de las columnas de df_clasificaciones para esto.

In [ ]:
df_clasificaciones.drop(columns=["Código segmento","Código Familia","Nombre Familia","Nombre Clase","Código Clase"], inplace=True)

### Instituciones que realizan licitaciones. 

Se obtuvo unb dataset con información de los organismos que han realizado lictaciones historicamente desde <https://datos-abiertos.chilecompra.cl/descargas/complementos>, se utilizará para determinar el sector del comprador.

In [ ]:
camino = path.join("Chile","Instituciones_compradoras.csv")
df_instituciones_compradoras = pd.read_csv(camino, sep=";", encoding="UTF-8")

In [ ]:
df_instituciones_compradoras.head()

,sector,Instituci�n,C�digo Instituci�n,RUT Unidad de Compra,C�digo Unidad de Compra,Unidad de Compra,Regi�n Unidad de Compra
0,MUNICIPALIDADES,I MUNICIPALIDAD DE AYSEN,100049,69.240.100-K,3701,I.MUNICIPALIDAD AYSEN-DIRECCION DE EDUCACION,Ays�n
1,MUNICIPALIDADES,I MUNICIPALIDAD DE AYSEN,100049,69.240.100-K,3702,I.MUNICIPALIDAD AYSEN-ADQUISICIONES,Ays�n
2,MUNICIPALIDADES,I MUNICIPALIDAD DE AYSEN,100049,60.920.753-1,3703,I.MUNICIPALIDAD AYSEN-LICEO POLITECNICO,Ays�n
3,MUNICIPALIDADES,I MUNICIPALIDAD DE PIRQUE,100072,69.072.200-3,3704,MUNICIPALIDAD DE PIRQUE - DAF,Metropolitana
4,MUNICIPALIDADES,I MUNICIPALIDAD DE PIRQUE,100072,69.072.200-3,3705,MUNICIPALIDAD DE PIRQUE - DOM,Metropolitana


In [ ]:
df_instituciones_compradoras.loc[df_instituciones_compradoras["C�digo Unidad de Compra"] == 5772]

,sector,Instituci�n,C�digo Instituci�n,RUT Unidad de Compra,C�digo Unidad de Compra,Unidad de Compra,Regi�n Unidad de Compra
2446,MUNICIPALIDADES,I MUNICIPALIDAD DE TIRUA,183037,69.160.700-3,5772,FINANZAS,B�o-B�o


In [ ]:
df_instituciones_compradoras.drop(columns=["Instituci�n","Regi�n Unidad de Compra", "RUT Unidad de Compra", "C�digo Instituci�n"], inplace=True)
df_instituciones_compradoras.rename(columns={"C�digo Unidad de Compra":"Codigo Unidad de Compra"}, inplace=True)
df_instituciones_compradoras.head()

,sector,C�digo Unidad de Compra,Unidad de Compra
0,MUNICIPALIDADES,3701,I.MUNICIPALIDAD AYSEN-DIRECCION DE EDUCACION
1,MUNICIPALIDADES,3702,I.MUNICIPALIDAD AYSEN-ADQUISICIONES
2,MUNICIPALIDADES,3703,I.MUNICIPALIDAD AYSEN-LICEO POLITECNICO
3,MUNICIPALIDADES,3704,MUNICIPALIDAD DE PIRQUE - DAF
4,MUNICIPALIDADES,3705,MUNICIPALIDAD DE PIRQUE - DOM


### Unión de dataframes.

In [ ]:
df_filtrado.info()

In [ ]:
df_awards.info()

In [ ]:
df_awards_items.info()

In [ ]:
df_awards_supplier.info()

In [ ]:
df_parties.info()

In [ ]:
df_parties_add.info()

In [ ]:
df_tender_item.info()

In [ ]:
df_tender_tenderers.info()

In [ ]:
df_clasificaciones.info()

### Award_items y clasificaciones.

In [ ]:
df_award_item_clasifi = df_awards_items.merge(df_clasificaciones, left_on="classification_id",right_on="Código Producto")
df_award_item_clasifi.drop(columns=[ "_link"], inplace=True)
df_award_item_clasifi.head()

### Awards y Awards_items

In [ ]:
df_award_and_items = df_awards.merge(df_award_item_clasifi, left_on="_link", right_on="_link_awards")
df_award_and_items.head()

In [ ]:
df_award_and_items.drop(columns=["_link_main_y", "_link_awards"],inplace=True)

In [ ]:
df_igualdad = pd.DataFrame(df_award_and_items["value_monetary_unit"] == df_award_and_items["unit_value_monetary_unit"])
indice = df_igualdad.loc[df_igualdad[0] == False].index

In [ ]:
df_award_and_items.iloc[indice]

In [ ]:
df_award_and_items.drop(indice, inplace=True)

In [ ]:
df_award_and_items.rename({"_link_main_x":"_link_main"},inplace=True)

In [ ]:
df_award_and_items.columns

### Awards y Awards_supplier

In [ ]:
df_award_and_supplier = df_awards.merge(df_awards_supplier, left_on="_link", right_on="_link_awards")
df_award_and_supplier.drop(columns=["_link_awards", "_link_main_y", "_link_y"], inplace=True)
df_award_and_supplier.head()

In [ ]:
df_award_and_supplier.info()

#### Tender_items y Clasificaciones.

In [ ]:
df_tender_item = df_tender_item.merge(df_clasificaciones, left_on="classification_id",right_on="Código Producto")
df_tender_item.drop(columns=["classification_scheme", "classification_id"], inplace=True)
df_tender_item.head()

### Awards y Tender_items_clasificaciones.

In [ ]:
df_items_tender_awards = df_award_and_items.merge(df_tender_item, left_on=["_link_main_x","classification_id","unit_name"], right_on=["_link_main","Código Producto","unit_name"], how="left")

In [ ]:
df_items_tender_awards.head()

In [ ]:
df_items_tender_awards.columns

In [ ]:
df_items_tender_awards.drop(columns=['_link', '_link_main','quantity_x', 'classification_id', "id_x"], inplace=True)
df_items_tender_awards.rename(columns={'_link_x':"_link_awards", '_link_main_x':"_link_main", 'date':"date_award", 'status':"award_status",'quantity_x':"quantity", 'unit_name_x':"unit_name", "id_y":"award_id"}, inplace=True)
df_items_tender_awards.head()

### Parties y Parties_additional_identifiers.

In [ ]:
df_parties_and_add = df_parties.merge(df_parties_add, left_on=["_link","id"], right_on=["_link_parties", "id"], how="left")
df_parties_and_add.head()

In [ ]:
df_parties_and_add.drop(columns=["_link_y", "_link_parties"], inplace=True)
df_parties_and_add.head()

In [ ]:
df_parties_and_add.info()

### Union de awards_supplier y parties.

In [ ]:
df_supplier_additional_info = df_award_and_supplier.merge(df_parties_and_add, left_on=["_link_main_x", "name"], right_on=["_link_main", "name"])
df_supplier_additional_info.info()

In [ ]:
df_supplier_additional_info.head()

In [ ]:
df_supplier_additional_info.drop(columns=["id_y", "_link_main", "_link_x_y"], inplace=True)
df_supplier_additional_info.rename(columns={"_link_main_x":"_link_main", "_link_x_x":"_link_awards", "id_x":"supplier_id"}, inplace=True)
df_supplier_additional_info.head()

## Unión de main con otros dataframes.

In [ ]:
df_main_buyer_info = df_filtrado.merge(df_parties_and_add, right_on=["id","_link_main"], left_on=["buyer_id","_link"], how="left")
df_main_buyer_info.info()

In [ ]:
df_main_buyer_info.head()

In [ ]:
df_main_buyer_info.drop(columns=["buyer_id", "tender_procuringEntity_name", "_link_main","rol 1" ,"rol 2","_link_x", "address_countryName","_link_x", "_link_main"], inplace=True)
df_main_buyer_info.rename(columns={"address_region":"buyer_region", "name":"buyer_name", "id":"buyer_id", "legalName":"buyer_legalName", "tender_procurementMethodDetails":"procurement_details", "tender_procurementMethod":"procurementMethod", "address_countryName":"buyer_country"},inplace=True)
df_main_buyer_info.head()

####  Merge de df_main_buyer con df_supplier_additional_info y df_awards

In [ ]:
df_main_buyer_suplier = df_main_buyer_info.merge(df_supplier_additional_info, left_on="_link", right_on="_link_main", suffixes=("main_", "_supplier"), how="left")
df_main_buyer_suplier.head()

In [ ]:
df_main_buyer_suplier.columns

In [ ]:
df_main_buyer_suplier[['estimated_cost_monetary_unit', 'buyer_id', 'buyer_name','buyer_region', 'buyer_legalName', '_link_awards', '_link_main', 'date','status', 'value_amount', 'value_monetary_unit', 'supplier_id', 'name','address_region', 'address_countryName', 'rol 1', 'rol 2', 'legalName']].head()

In [ ]:
df_main_buyer_suplier.rename(columns={'date':"date_award",'status':"award_status", 'value_amount':"tender_value", 'value_monetary_unit':"tender_monetary_unit", 'name':"supplier_name",'address_region':"supplier_region", 'address_countryName':"supplier_country", 'legalName':"supplier_legal_name"}, inplace=True)
df_main_buyer_suplier.head()

In [ ]:
df_main_buyer_suplier.drop(columns=['rol 1', 'rol 2', '_link_awards', '_link_main'], inplace=True)

In [ ]:
df_main_buyer_suplier.head()

In [ ]:
df_main_award = df_main_buyer_info.merge(df_tender_item, left_on="_link", right_on="_link_main", how="left")
df_main_award.head()

### main_buyer y df_awards.

In [ ]:
df_main_buyer_award = df_main_buyer_info.merge(df_award_item_clasifi[["_link_awards",]])

### main_buyer, awards y tenderers.

In [ ]:
df_tenderers = df_tender_tenderers.merge(df_parties_and_add, left_on=["_link_main","id"], right_on=["_link_main", "id"], how="left")
df_tenderers.head()

In [ ]:
df_main_buyer_award= df_main_buyer_info.merge()

In [ ]:
df_main_tenderers = df_main_buyer_info.merge(df_tenderers, left_on="_link", right_on = "_link_main", how="left")


In [ ]:
df_main_tenderers.head()

In [ ]:
df_main_tenderers.columns

In [ ]:
df_main_tenderers["Has_tender"] = df_main_tenderers.index.map(lambda x: 1 if df_main_tenderers.loc[x]["rol 1"] == "tenderer" else 0)
df_main_tenderers["is_supplier"] = df_main_tenderers.index.map(lambda x: 1 if df_main_tenderers.loc[x]["rol 2"] == "supplier" else 0)


In [ ]:
df_main_tenderers.drop(columns=["_link_main", "_link_y","name_x"],inplace=True)

In [ ]:
df_main_tenderers.rename(columns={"_link_x":"_link", "name_y":"tender_name", "address_region":"tenderer_region", "address_countryName":"tenderer_country","legalName":"tenderer_legalName"},inplace=True)
df_main_tenderers.head()

In [ ]:
df_main_tenderers.groupby(by='tenderer_region').agg({'Has_tender':'sum'}).reset_index()

In [ ]:
df_main_tenderers.groupby(by='tenderer_legalName').agg({'Has_tender':'sum'}).sort_values(by="Has_tender", ascending=False)

In [ ]:
df_main_tenderers.groupby(by='tenderer_legalName').agg({'is_supplier':'sum'}).sort_values(by="is_supplier", ascending=False)

In [ ]:
df_main_tenderers.groupby(by='tenderer_region').agg({'is_supplier':'sum'}).sort_values(by="is_supplier", ascending=False)